In [1]:
import glob
import os
from subprocess import check_output
import argparse
import re

In [150]:
def make_file(sub_id, ses_id, trial_id, output_dir, task,data_dict):
    with open(os.path.join('/projects/niblab/bids_projects/Experiments/Bevel/derivatives/design_files/level1_betaseries.fsf'),'r') as infile:
        tempfsf=infile.read()
        #
        if not os.path.exists(os.path.join(output_dir, "design_files")):
            os.makedirs(os.path.join(output_dir, "design_files"))
        print(output_dir)
        design_fileout = os.path.join(output_dir, "design_files/%s_%s_%s_feat1.fsf"%(sub_id, ses_id, trial_id))
        out_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["OUTPUT"]
        func_param = data_dict[sub_id][task]["FUNCRUN"]
        confound_param = data_dict[sub_id][task]["CONFOUND"]
        trial_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["TRIAL"]
        nuis_param = data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id]["NUIS"]

        tempfsf = tempfsf.replace("OUTPUT", out_param)
        tempfsf = tempfsf.replace("FUNCRUN", func_param) 
        tempfsf = tempfsf.replace("CONFOUND", confound_param)
        tempfsf = tempfsf.replace("TRIAL", trial_param)
        tempfsf = tempfsf.replace("NUIS", nuis_param)

        for i in range(6):
            moco = data_dict[sub_id][task]["MOCO%i"%i]
            tempfsf = tempfsf.replace("MOCO%i"%i, moco)
        try:
            with open(design_fileout,'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        except:
            print("BAD SUBJECT ", sub_id)
        infile.close()
        
        

In [154]:
def create_fsf(input_dir, deriv_dir, ses_id):
    
    ses_id=ses_id
    data_dict= {}
    # start loop -- looping through subjects
    subject_list = glob.glob(os.path.join('/projects/niblab/bids_projects/Experiments/ChocoData/derivatives', 'sub-*/%s'%ses_id))
    for sub_path in subject_list[:3]:
        sub_id=sub_path.split("/")[-2]
        
        functional_tasks = glob.glob(os.path.join(sub_path, 'func/*milkshake*preproc_brain.nii.gz'))
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
                
        
        for functional in functional_tasks: # SECOND LOOP -- looping through RUNS
            task=functional.split("/")[-1].split("_")[2].split("-")[1]
            analysis_folder=os.path.join('/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/%s/%s'%(sub_id,ses_id), "analysis")
            #print(analysis_folder)
            output_dir = os.path.join(analysis_folder, 'beta/task-%s'%task)
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            confound_file = os.path.join(sub_path, "func/motion_assessment/%s_%s_task-%s_bold_space-MNI152NLin2009cAsym_preproc_brain_confound.txt"%(sub_id,ses_id, task))
                
            data_dict[sub_id][task] = {
                "TRIALS" : { },
                "CONFOUND" : confound_file,
                "FUNCRUN" : functional
              }           
            
            for i in range(6):
                motcor=os.path.join(sub_path, 'func','motion_assessment', 'motion_parameters','%s_%s_task-%s_moco%s.txt'%(sub_id,ses_id, task,i))
                data_dict[sub_id][task]['MOCO%i'%i] = motcor
            
            
            subj_trials = sorted(glob.glob('/projects/niblab/experiments/chocolate_milkshake/data/evs/%s*trial*.txt'%task.replace('milkshake', 'mk')))
            
            if not subj_trials:
                pass
            else:
                for trial_file in subj_trials:
                    _id = sub_id.split("-")[1]
                    _id = _id[1:]
                    trial_id = trial_file.split("/")[-1].split(".")[0]
                    #print(trial_id)
                    nuis_file = os.path.join('/projects/niblab/experiments/chocolate_milkshake/data/evs', '%s.txt'%trial_id.replace('trial', 'nuis'))
                    #print(nuis_file)
                    fileout = os.path.join(output_dir, "%s_%s_%s"%(sub_id, ses_id,trial_id))
                    #print(fileout)
                    data_dict[sub_id][task]["TRIALS"]["TRIAL%s"%trial_id] = {"TRIAL" : trial_file, "NUIS": nuis_file, "OUTPUT" : fileout}
                    make_file(sub_id, ses_id, trial_id,output_dir, task, data_dict)
                    
                    
    return(data_dict)

In [155]:
def set_paths():
    #global input_dir
    #global deriv_dir
    print("[INFO] setting directory paths")
    input_dir =  '/projects/niblab/experiments/chocolate_milkshake'
        
    deriv_dir= os.path.join(input_dir, 'bids/derivatives')

    return input_dir, deriv_dir



In [156]:
def main():
    ses_id='ses-1'
    input_dir, deriv_dir=set_paths()
    data_dict=create_fsf(input_dir, deriv_dir,ses_id)
    
main()


[INFO] setting directory paths
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/t

/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-001/ses-1/analysis/beta/task-milkshakeB
/projects/niblab

/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-004/ses-1/analysis/beta/task-milkshakeA
/projects/niblab

/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeC
/projects/niblab

/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeB
/projects/niblab/experiments/chocolate_milkshake/bids/derivatives/preprocessed/sub-005/ses-1/analysis/beta/task-milkshakeB
